#### Import Statements

In [3]:
# Import statements
import sqlite3
import pandas as pd 
from tabulate import tabulate
from openpyxl import Workbook

#### Create database and insert records from csv files

In [4]:
# Connect to the SQLite database
connection = sqlite3.connect('hospital.db')
cursor = connection.cursor()

# Delete tables if exist  
cursor.execute(''' DROP TABLE IF EXISTS patients;  ''')
cursor.execute(''' DROP TABLE IF EXISTS doctors;  ''')
cursor.execute(''' DROP TABLE IF EXISTS admissions;  ''')
cursor.execute(''' DROP TABLE IF EXISTS province_names;  ''')

# Create tables based on the schema 
cursor.execute(''' CREATE TABLE patients (
                    patient_id INTEGER PRIMARY KEY,
                    first_name VARCHAR(30),
                    last_name VARCHAR(30),
                    gender CHAR(1),
                    birth_date DATE,
                    city VARCHAR(30),
                    province_id CHAR(2),
                    allergies VARCHAR(80),
                    height DECIMAL(3,0),
                    weight DECIMAL(4,0)    ); ''')

cursor.execute(''' CREATE TABLE doctors (
                    doctor_id INTEGER PRIMARY KEY,
                    first_name VARCHAR(30),
                    last_name VARCHAR(30),
                    speciality VARCHAR(25)   ); ''')

cursor.execute(''' CREATE TABLE admissions (
                    patient_id INT,
                    admission_date DATE,
                    discharge_date DATE,
                    diagnosis VARCHAR(50),
                    attending_doctor_id INT,
                    FOREIGN KEY(patient_id) REFERENCES patients(patient_id),
                    FOREIGN KEY(attending_doctor_id) REFERENCES doctors(doctor_id)    );  ''')

cursor.execute(''' CREATE TABLE province_names (
                    province_id CHAR(2) PRIMARY KEY,
                    province_name VARCHAR(30)    );  ''')

# Load data from CSV files into tables
# Load patients data
patients_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\patients.csv')
patients_df.to_sql('patients', connection, if_exists='append', index=False)

# Load doctors data
doctors_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\doctors.csv')
doctors_df.to_sql('doctors', connection, if_exists='append', index=False)

# Load admissions data
admissions_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\admissions.csv')
admissions_df.to_sql('admissions', connection, if_exists='append', index=False)

# Load province names data
province_names_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\province_names.csv')
province_names_df.to_sql('province_names', connection, if_exists='append', index=False)


13

### Functions

In [5]:
# Display table from hospital.db 
def showTableFromDB(table_name):
    query = f'SELECT * FROM {table_name} '
    cursor.execute(query)

    # Display first 5 records
    tableRecords = cursor.fetchmany(5)
    
    # Get column names from the cursor
    column_name = [ description[0] for description in cursor.description ]

    # Display in table format
    print( tabulate(tableRecords, headers= column_name, tablefmt= 'simple') )

In [6]:
# Display result of query
def displayOutput(cursor):
    result = cursor.fetchall()

    # Calculate number of records
    print('Number of Records - ', len(result) )

    # Fetch column names
    column_name = [ description[0] for description in cursor.description ]

    # Print the result in table format
    print( tabulate(result, headers= column_name, tablefmt= 'simple') )

#### To load required data in an Excel file : <br> Call function after `cursor.execute()`

> cursor.execute(''' QUERY ''') <br>
> export_query_to_excel(cursor, 'Output.xlsx') 

In [7]:
# Function to laod data in excel file 
def export_query_to_excel(cursor, file_name ): 
    results = cursor.fetchall()
    
    # Get column names from the cursor
    column_names = [description[0] for description in cursor.description]
    
    # Create a workbook and add a worksheet
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = 'Result'
    
    # Write column headers
    sheet.append(column_names)
    
    # Write rows (Records)
    for row in results:
        sheet.append(row)
    
    # Save the workbook to the specified output file
    workbook.save(file_name)
    print(f'Data Exported Successfully to {file_name}')

### Questions from Management Team :

#### 1. Can you provide the first and last names of all male patients ?

In [8]:
cursor.execute(''' SELECT first_name, last_name, gender 
                    FROM patients 
                    WHERE gender LIKE 'M'; ''')

displayOutput(cursor)

Number of Records -  2468
first_name     last_name         gender
-------------  ----------------  --------
Donald         Waterfield        M
Mickey         Baasha            M
Jiji           Sharma            M
Blair          Diaz              M
Charles        Wolfe             M
Thomas         ONeill            M
Sonny          Beckett           M
Cedric         Coltrane          M
Hank           Spencer           M
Rick           Bennett           M
Woody          Bashir            M
Tom            Halliwell         M
John           West              M
Jon            Doggett           M
Nino           Andrews           M
John           Farley            M
Jim            Thomas            M
Roderick       Payne             M
Sam            Threep            M
Roland         Edge              M
Don            Wiggum            M
Anthony        Maxwell           M
James          Sullivan          M
Charles        Andonuts          M
Don            Edwards           M
Stone          Cu

#### 2. Who are the patients that do not have any allergies recorded ?

In [9]:
cursor.execute(''' SELECT first_name, last_name FROM patients
                    WHERE allergies IS NULL; ''')

displayOutput(cursor)

Number of Records -  2059
first_name     last_name
-------------  ----------------
Donald         Waterfield
Blair          Diaz
Thomas         ONeill
Sonny          Beckett
Cedric         Coltrane
Hank           Spencer
Sara           di Marco
Amy            Leela
Rachel         Winterbourne
John           West
Jon            Doggett
Angel          Edwards
Brodie         Beck
Nea            Kobayakawa
Laura          Halliwell
Lucille        Maureen
Roderick       Payne
Bertha         Crowley
Don            Wiggum
Anthony        Maxwell
Anne           Marlowe
Charles        Andonuts
Don            Edwards
Cross          Gordon
Harry          Kuramitsu
Gary           Ramotswe
Phil           Beckett
Gala           Littlefield
Gregory        James
Molly          Jackson
Rose           Spellman
Will           Manu
Rocky          Donahue
Mary           Spellman
Didi           Dean
Lily           Crusher
Billy          Jones
Vicki          Babbling
Temple         Marlowe
Jackie         Loveg

#### 3. Could you list the patients whose names start with the letter 'C' ?

In [10]:
cursor.execute(''' SELECT * FROM patients
                    WHERE first_name LIKE 'C%';  ''')

displayOutput(cursor) 

Number of Records -  302
  patient_id  first_name    last_name         gender    birth_date    city            province_id    allergies                   height    weight
------------  ------------  ----------------  --------  ------------  --------------  -------------  ------------------------  --------  --------
           5  Charles       Wolfe             M         2017-11-19    Orillia         ON             Penicillin                      47        10
          10  Cedric        Coltrane          M         1961-11-10    Toronto         ON                                            157        61
          39  Charles       Andonuts          M         2016-05-20    Hamilton        ON                                             62        15
          45  Cross         Gordon            M         2009-03-20    Ancaster        ON                                            125        53
          63  Calleigh      Dean              F         2003-07-06    Hamilton        ON           

#### 4. Can we get a list of patients weighing between 100 and 120 units ?

In [11]:
cursor.execute(''' SELECT first_name, last_name 
                    FROM patients
                    WHERE weight BETWEEN 100 AND 120;  ''')

displayOutput(cursor)

Number of Records -  952
first_name    last_name
------------  ----------------
Jiji          Sharma
Blair         Diaz
Thomas        ONeill
Sonny         Beckett
Tom           Halliwell
Jon           Doggett
Angel         Edwards
John          Farley
Temple        Russert
Don           Edwards
Gary          Ramotswe
Phil          Chester
Woody         Riviera
Rocky         Donahue
Shiro         Whale
Didi          Dean
Catherine     Minoru
Danny         Hardbroom
Lily          Vargas
Reginald      Jackson
Phoebe        Bell
Ann           Blocksberg
Sam           Spratt
Prue          Cleever
Zoe           Miller
Mendy         Strate
Hal           Riviera
John          Stape
Alice         Taylor
Salvatore     Neutron
Thomas        Fortune
Arnold        Love
Vin           Smyth
Kenny         Skelton
Mike          Madison
Keziah        Brearly
Jeff          Tyrell
Daneel        Carpenter
Robert        Page
Wanda         Redfield
Brooke        Bowles
Hoshi         Johnson
Joanna        Coo

#### 5. Could you update records so that patients without allergy information show as 'NKA' (No Known Allergies) ?

In [12]:
cursor.execute(''' UPDATE patients
                    SET allergies = 'NKA'
                    WHERE allergies IS NULL; ''')

# Saving changes in databse
connection.commit()

showTableFromDB('patients') 

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


#### 6. Can we display the full name of each patient ?  

In [13]:
cursor.execute(''' SELECT CONCAT(first_name, ' ', last_name) AS full_name
                    FROM patients;  ''')

displayOutput(cursor)

Number of Records -  4530
full_name
---------------------------
Donald Waterfield
Mickey Baasha
Jiji Sharma
Blair Diaz
Charles Wolfe
Sue Falcon
Thomas ONeill
Sonny Beckett
Sister Spitzer
Cedric Coltrane
Hank Spencer
Sara di Marco
Daphne Seabright
Rick Bennett
Amy Leela
Woody Bashir
Tom Halliwell
Rachel Winterbourne
John West
Jon Doggett
Angel Edwards
Brodie Beck
Beanie Foster
Nino Andrews
John Farley
Nea Kobayakawa
Laura Halliwell
Lucille Maureen
Jim Thomas
Roderick Payne
Sam Threep
Bertha Crowley
Roland Edge
Don Wiggum
Anthony Maxwell
James Sullivan
Anne Marlowe
Kelly Hamilton
Charles Andonuts
Temple Russert
Don Edwards
Alice Donovan
Stone Cutting
Neil Allan
Cross Gordon
Phoebe Bigelow
Harry Kuramitsu
Gary Ramotswe
Phil Chester
Phil Beckett
Gala Littlefield
Gregory James
Bruno Wolfe
Molly Jackson
Rose Spellman
Will Manu
Woody Riviera
Georgette Gallagher
Rocky Donahue
Roland Murphy
Mary Spellman
Shiro Whale
Calleigh Dean
Didi Dean
Haiku Alpert
Joseph Justin
Catherine Minoru
Danny Hardb

#### 7. Could we see each patient’s name along with their full province name ?

In [14]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [15]:
showTableFromDB('province_names')

province_id    province_name
-------------  -------------------------
AB             Alberta
BC             British Columbia
MB             Manitoba
NB             New Brunswick
NL             Newfoundland and Labrador


In [16]:
cursor.execute(''' SELECT first_name, last_name, province_name
                    FROM patients AS p 
                    INNER JOIN province_names AS pn 
                    ON p.province_id = pn.province_id ;  ''')

displayOutput(cursor)

Number of Records -  4530
first_name     last_name         province_name
-------------  ----------------  -------------------------
Donald         Waterfield        Ontario
Mickey         Baasha            Ontario
Jiji           Sharma            Ontario
Blair          Diaz              Ontario
Charles        Wolfe             Ontario
Sue            Falcon            Ontario
Thomas         ONeill            Ontario
Sonny          Beckett           Nova Scotia
Sister         Spitzer           Ontario
Cedric         Coltrane          Ontario
Hank           Spencer           Ontario
Sara           di Marco          Ontario
Daphne         Seabright         Ontario
Rick           Bennett           Ontario
Amy            Leela             Ontario
Woody          Bashir            Ontario
Tom            Halliwell         Ontario
Rachel         Winterbourne      Ontario
John           West              Ontario
Jon            Doggett           Ontario
Angel          Edwards           Ontario
Bro

#### 8. How many of our patients were born in 2010 ?

In [17]:
cursor.execute(''' SELECT COUNT(birth_date) AS Count
                    FROM patients 
                    WHERE strftime('%Y', birth_date) = '2010';  ''')

displayOutput(cursor)

Number of Records -  1
  Count
-------
     55


#### 9. Find the tallest patient, showing their full name and height.

In [18]:
cursor.execute(''' SELECT first_name, last_name, MAX(height)
                    FROM patients ''')

displayOutput(cursor)

Number of Records -  1
first_name    last_name      MAX(height)
------------  -----------  -------------
Sam           Haruko                 226


#### 10. Could you retrieve all the details for patients with IDs 1, 45, 534, 879, and 1000 ?

In [19]:
cursor.execute(''' SELECT * 
                    FROM patients
                    WHERE patient_id IN (1, 45, 534, 879, 1000);  ''')

displayOutput(cursor)

Number of Records -  5
  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
          45  Cross         Gordon       M         2009-03-20    Ancaster  ON             NKA               125        53
         534  Don           Zatara       M         2008-01-11    Timmins   ON             NKA               136        67
         879  Orla          Shawn        F         1967-09-24    Sarnia    ON             Penicillin        149        65
        1000  Rick          Williams     M         1975-04-13    Hamilton  ON             Penicillin        176       127


#### 11. What is the total count of admissions to date ?

In [20]:
cursor.execute(''' SELECT count(*) AS total_admissions
                    FROM admissions;  ''')

displayOutput(cursor)

Number of Records -  1
  total_admissions
------------------
              5067


#### 12. Who were admitted and discharged on the same day ?

In [21]:
cursor.execute(''' SELECT *
                    FROM admissions
                    WHERE admission_date = discharge_date;  ''')

displayOutput(cursor)

Number of Records -  481
  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           9  2018-12-31        2018-12-31        Ruptured Appendicitis                                                  19
          10  2019-02-27        2019-02-27        Lower Quadrant Pain                                                    27
          17  2019-03-04        2019-03-04        Diabetes Mellitus                                                       9
          28  2019-03-30        2019-03-30        Cancer Of The Stomach                                                  26
          31  2018-09-26        2018-09-26        Cardiovascular Disease                                   

#### 13. How many admissions has patient ID 579 had so far ?

In [22]:
cursor.execute(''' SELECT patient_id, COUNT(*) AS total_admissions
                    FROM admissions
                    WHERE patient_id = 579;  ''')

displayOutput(cursor)

Number of Records -  1
  patient_id    total_admissions
------------  ------------------
         579                   2


#### 14. Could you list the unique cities in Nova Scotia where our patients live ?

In [23]:
cursor.execute(''' SELECT DISTINCT city AS unique_cities
                    FROM patients
                    WHERE province_id = 'NS';  ''')

displayOutput(cursor)

Number of Records -  3
unique_cities
---------------
Port Hawkesbury
Halifax
Inverness


#### 15. Could you find patients First name, Last name, Birth Date who are over 160 cm in height and over 70 kg in weight ?

In [24]:
cursor.execute(''' SELECT first_name, last_name, birth_date
                    FROM patients
                    WHERE height > 160 AND weight > 70;  ''')

displayOutput(cursor)

Number of Records -  2091
first_name     last_name         birth_date
-------------  ----------------  ------------
Mickey         Baasha            1981-05-28
Jiji           Sharma            1957-09-05
Blair          Diaz              1967-01-07
Thomas         ONeill            1993-01-31
Sonny          Beckett           1952-12-11
Sister         Spitzer           1966-10-15
Rick           Bennett           1977-01-27
Amy            Leela             1977-06-25
Tom            Halliwell         1987-08-01
Rachel         Winterbourne      1966-04-26
Jon            Doggett           1951-12-25
Angel          Edwards           1975-08-22
Nino           Andrews           2001-04-21
John           Farley            1956-09-12
Lucille        Maureen           1953-08-22
Sam            Threep            1982-06-16
Roland         Edge              1967-09-25
Don            Wiggum            2002-11-15
Anne           Marlowe           1996-09-17
Temple         Russert           1953-05-11
Don 

#### 16. Who in Hamilton has recorded allergies ?

In [25]:
cursor.execute(''' SELECT first_name, last_name, allergies
                    FROM patients
                    WHERE city = 'Hamilton' AND allergies != 'NKA';  ''')

displayOutput(cursor)

Number of Records -  1046
first_name    last_name         allergies
------------  ----------------  ------------------------
Jiji          Sharma            Penicillin
Tom           Halliwell         Ragweed
Nino          Andrews           Peanuts
John          Farley            Gluten
Sam           Threep            Sulpha
Temple        Russert           Hay Fever
Alice         Donovan           Penicillin
Stone         Cutting           Codeine
Phil          Chester           Penicillin
Roland        Murphy            Sulfa Drugs
Shiro         Whale             Penicillin
Calleigh      Dean              Penicillin
Lily          Vargas            Codeine
Kazuki        Ranall            Penicillin
Mike          Mars              Penicillin
Matt          Goergen           Penicillin
Ann           Blocksberg        Penicillin
Zoe           Miller            Honey
Gene          Edwards           Sulphate
Emmett        Erin              Penicillin
Olwyn         Chino             Tetracycli

#### 17. List all unique patient birth years in ascending order.

In [26]:
cursor.execute(''' SELECT DISTINCT strftime('%Y', birth_date) AS birth_year
                    FROM patients
                    ORDER BY birth_date ASC;  ''')

displayOutput(cursor)

Number of Records -  93
  birth_year
------------
        1918
        1923
        1925
        1926
        1927
        1928
        1929
        1931
        1933
        1934
        1936
        1937
        1938
        1939
        1940
        1941
        1942
        1943
        1944
        1945
        1946
        1947
        1948
        1949
        1950
        1951
        1952
        1953
        1954
        1955
        1956
        1957
        1958
        1959
        1960
        1961
        1962
        1963
        1964
        1965
        1966
        1967
        1968
        1969
        1970
        1971
        1972
        1973
        1974
        1975
        1976
        1977
        1978
        1979
        1980
        1981
        1982
        1983
        1984
        1985
        1986
        1987
        1988
        1989
        1990
        1991
        1992
        1993
        1994
        1995
        1996
        1997
        1998
 

In [27]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


#### 18. Are there any names that appear only once in our patient records ? For example, if two or more people are named 'John' in the first_name column then don't include their name in the output list. If only 1 person is named 'Leo' then include them in the output.

In [28]:
cursor.execute(''' SELECT first_name
                    FROM patients
                    GROUP BY first_name
                    HAVING COUNT(first_name) = 1;  ''')

displayOutput(cursor)

Number of Records -  319
first_name
------------
Abby
Adelaide
Adelia
Akira
Albert
Aldo
Alec
Alicia
Allan
Alpa
Amane
Amitabh
Angel
Angelina
Angus
Anne
Anton
Antonia
Anubis
Apolline
April
Ariel
Asa
Astrid
Asuka
Audrey
Aurora
Avery
Bathilda
Bathsheda
Beverly
Bianca
Bibi
Bigby
Bob
Borat
Brick
Brick Top
Brodie
Buffy
Buford
Bunk
Burr
Byron
Caitlin
Calvin
Carla
Carlton
Carmel
Carmen
Cary
Cat
Chan
Cherry
China
Cho
Christina
Christy
Chuny
Cindy
Clancy
Clarence
Claudette
Clay
Clayton
Colleen
Connie
Cooper
Crematia
Cristina
Cully
Cuthbert
Cyrus
Daisuke
Daisy
Dave
Davina
Debbie
Debra
Delia
Denise
Deunan
Diana
Dickie
Dion
Dorian
Drogulous
Duncan
Dusty
Dwight
Edgar
Eglantine
Eleanor
Ellie
Ellis
Elly
Elsie
Emerson
Enid
Epiphany
Eric
Erica
Erika
Ervin
Escobar
Fancy
Frankie
Frau
Frederico
Gabriel
Gabrielle
Gemma
Georgia
Gina
Ginevra
Gordon
Gus
Gwenog
Gytha
Haley
Hans
Hari
Harvey
Henri
Herb
Hermione
Hildegarde
Hillary
Hiroshi
Holi
Holland
Horatio
Ichabod
Isaac
Isabel
Isabella
Jabba
Janine
Janus
Jasmine

#### 19. Could you find patients whose first names start and end with ‘S’ and have at least six letters ?

In [29]:
cursor.execute(''' SELECT patient_id, first_name
                    FROM patients
                    WHERE first_name LIKE 'S%S' AND LENGTH(first_name) >= 6;  ''')

displayOutput(cursor)

Number of Records -  11
  patient_id  first_name
------------  ------------
         496  Spiros
         629  Spiros
         648  Stanislaus
        1273  Stanislaus
        1789  Seamus
        1926  Stanislaus
        1996  Stanislaus
        2258  Spiros
        2378  Stanislaus
        2771  Stanislaus
        3487  Stanislaus


#### 20. Can we list patients diagnosed with ‘Dementia’ ?

In [30]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [31]:
showTableFromDB('admissions')

  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-11-06        2018-11-08        Ovarian Dermoid-Cyct                                                   21
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           3  2019-01-24        2019-01-29        Cardiac Arrest                                                          2
           3  2018-10-21        2018-10-27        Congestive Heart Failure                                                8
           6  2018-06-13        2018-06-15        Asthma Exacerbation                                                     3


In [32]:
cursor.execute(''' SELECT p.patient_id, p.first_name, p.last_name
                    FROM patients AS p
                    LEFT JOIN admissions AS a 
                    ON p.patient_id = a.patient_id
                    WHERE diagnosis = 'Dementia';  ''')

displayOutput(cursor)

Number of Records -  26
  patient_id  first_name    last_name
------------  ------------  -----------
         160  Miranda       Delacour
         178  David         Bustamonte
         207  Matt          Celine
         613  Jaki          Granger
         836  Montana       Vimes
         924  Simon         Spellman
        1201  Irene         Murphy
        1264  Jillian       Valentine
        1402  Kathryn       Hallow
        1491  Doris         McGrew
        1585  Alex          Cantropus
        1749  Alejandro     Mellie
        1798  Sister        Trenton
        1844  Jadu          Wilson
        1938  Anthony       Anneke
        2292  Jessica       Talbot
        2532  Lynn          Willing
        2689  Kurt          Jones
        2834  Juliet        Harkness
        3375  Agnes         Barkley
        3496  Harry         Lucas
        3555  Jack          Caspar
        3842  Claudia       Drew
        4256  Rita          Wolfe
        4357  Alex          Moss
        440

#### 21. List all patient first name ordered by name length, then alphabetically.

In [33]:
cursor.execute(''' SELECT first_name 
                    FROM patients
                    ORDER BY LENGTH(first_name) ASC, first_name ASC;  ''')

displayOutput(cursor)

Number of Records -  4530
first_name
-------------
Al
Al
Al
Al
Al
Al
Al
Bo
Bo
Bo
Bo
Bo
Bo
Cy
Cy
Ed
Ed
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Ri
Te
Te
Abe
Abe
Abe
Abe
Abe
Abi
Abi
Abi
Aki
Aki
Aki
Aki
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Ana
Ana
Ana
Ana
Ana
Ana
Ann
Ann
Ann
Ann
Ann
Ann
Ann
Art
Art
Art
Art
Art
Art
Art
Art
Art
Asa
Aya
Aya
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Bob
Cal
Cal
Cal
Cal
Cal
Cat
Cho
Dan
Dan
Dan
Dan
Dan
Dol
Dol
Dol
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Ema
Ema
Ema
Eva
Eva
Eva
Eva
Eva
Eva
Eva
Eve
Eve
Eve
Fin
Fin
Fin
Fiz
Fiz
Fiz
Fiz
Fiz
Gil
Gil
Gil
Gus
Guy
Guy
Guy
Guy
Guy
Guy
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Ian
Ian
Ian
Ivy
Ivy
Ivy
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jen
Jen
Jen
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jin
Jin
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Jon
Jon
Jon
Jon
Ka

#### 22. What is the count of male and female patients ? Display them side by side.

In [34]:
cursor.execute(''' SELECT 
                    SUM(gender = 'M') AS male_count,
                    SUM(gender = 'F') AS female_count
                    FROM patients;  ''')

displayOutput(cursor)

Number of Records -  1
  male_count    female_count
------------  --------------
        2468            2062


#### 23. List patients allergic to Penicillin or Morphine, ordered by allergy type and name.

In [35]:
cursor.execute(''' SELECT first_name, last_name, allergies
                    FROM patients
                    WHERE allergies IN ('Penicillin', 'Morphine')
                    ORDER BY allergies ASC, first_name ASC, last_name ASC;  ''')

displayOutput(cursor)

Number of Records -  1104
first_name    last_name         allergies
------------  ----------------  -----------
Briareos      Hayes             Morphine
Christine     Argyros           Morphine
Griselda      Hopper            Morphine
Henry         Huang             Morphine
Janice        Redfield          Morphine
Jesse         Guarnaccia        Morphine
Joel          Takata            Morphine
Jon           Guarnaccia        Morphine
Juan          Davies            Morphine
Kelly         Landsman          Morphine
Kent          Jacobs            Morphine
Mickey        Yamamoto          Morphine
Norman        Scott             Morphine
Patricia      Newbury           Morphine
Patty         Brady             Morphine
Raylan        Mallory           Morphine
Sarah         Pike              Morphine
Sarah Jane    Jones             Morphine
Temple        Starsmore         Morphine
Tugg          Travis            Morphine
Tyra          Shannon           Morphine
Uri           Falcon       

#### 24. Are there any patients admitted multiple times for the same diagnosis ?

In [36]:
cursor.execute(''' SELECT patient_id, diagnosis 
                    FROM admissions 
                    GROUP BY patient_id, diagnosis
                    HAVING COUNT(*) > 1;  ''')

displayOutput(cursor)

Number of Records -  11
  patient_id  diagnosis
------------  ---------------------------------
         137  Pregnancy
         320  Pneumonia
        1577  Congestive Heart Failure
        2004  Left Shoulder Rotator Cuff Repair
        2859  Severed Spine At C3
        3012  Appendicitis
        3367  Pyelonephritis
        3468  Congestive Heart Failure
        4083  Congestive Heart Failure
        4121  Congestive Heart Failure
        4363  Congestive Heart Failure


#### 25. List of city and the total number of patients in the city. Order from most to least patients and then by city name ascending.

In [37]:
cursor.execute(''' SELECT city, COUNT(*) AS total_number_of_patients
                    FROM patients
                    GROUP BY city
                    ORDER BY COUNT(*) DESC, city ASC;  ''')

displayOutput(cursor)

Number of Records -  93
city               total_number_of_patients
---------------  --------------------------
Hamilton                               1938
Toronto                                 317
Burlington                              276
Brantford                               147
Ancaster                                117
Stoney Creek                            107
Cambridge                                79
Dundas                                   79
Milton                                   65
Paris                                    58
Grimsby                                  55
Timmins                                  53
Ottawa                                   51
Halifax                                  50
Oakville                                 44
Sarnia                                   43
Fruitland                                42
Barrie                                   39
Delhi                                    37
St. Catharines                           35
Walnut G

In [38]:
cursor.execute(''' SELECT *
                    FROM patients;  ''')

displayOutput(cursor)

Number of Records -  4530
  patient_id  first_name     last_name         gender    birth_date    city             province_id    allergies                   height    weight
------------  -------------  ----------------  --------  ------------  ---------------  -------------  ------------------------  --------  --------
           1  Donald         Waterfield        M         1963-02-12    Barrie           ON             NKA                            156        65
           2  Mickey         Baasha            M         1981-05-28    Dundas           ON             Sulfa                          185        76
           3  Jiji           Sharma            M         1957-09-05    Hamilton         ON             Penicillin                     194       106
           4  Blair          Diaz              M         1967-01-07    Hamilton         ON             NKA                            191       104
           5  Charles        Wolfe             M         2017-11-19    Orillia        

#### 26. Can we show everyone affiliated with the hospital as either ‘Patient’ or ‘Doctor’ ?

In [39]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [40]:
showTableFromDB('doctors')

  doctor_id  first_name    last_name    speciality
-----------  ------------  -----------  -------------------------
          1  Claude        Walls        Internist
          2  Joshua        Green        Cardiologist
          3  Miriam        Tregre       General Surgeon
          4  James         Russo        Obstetrician/Gynecologist
          5  Scott         Hill         Gastroenterologist


In [41]:
cursor.execute(''' SELECT first_name, last_name, 'Patient' AS role FROM patients
                    UNION ALL
                   SELECT first_name, last_name, 'Doctor' AS role FROM doctors;  ''')

displayOutput(cursor)

Number of Records -  4557
first_name     last_name         role
-------------  ----------------  -------
Donald         Waterfield        Patient
Mickey         Baasha            Patient
Jiji           Sharma            Patient
Blair          Diaz              Patient
Charles        Wolfe             Patient
Sue            Falcon            Patient
Thomas         ONeill            Patient
Sonny          Beckett           Patient
Sister         Spitzer           Patient
Cedric         Coltrane          Patient
Hank           Spencer           Patient
Sara           di Marco          Patient
Daphne         Seabright         Patient
Rick           Bennett           Patient
Amy            Leela             Patient
Woody          Bashir            Patient
Tom            Halliwell         Patient
Rachel         Winterbourne      Patient
John           West              Patient
Jon            Doggett           Patient
Angel          Edwards           Patient
Brodie         Beck              P

#### 27. List of all allergies by popularity.

In [42]:
cursor.execute(''' SELECT allergies, COUNT(*) AS popularity
                    FROM patients
                    GROUP BY allergies
                    HAVING allergies != 'NKA'
                    ORDER BY popularity DESC;  ''')

displayOutput(cursor)

Number of Records -  63
allergies                   popularity
------------------------  ------------
Penicillin                        1082
Codeine                            305
Sulfa                              157
ASA                                 99
Sulfa Drugs                         71
Peanuts                             52
Iodine                              48
Tylenol                             42
Bee Stings                          40
Valporic Acid                       38
Tetracycline                        34
Wheat                               33
Peanut Butter                       29
Micropore Tape                      27
Strawberries                        25
Hay Fever                           25
Almonds                             23
Sulphur                             22
Morphine                            22
Tape                                20
Salicylic Acid                      20
Honey                               20
Penicillin Environmental            18
T

#### 28. Could we get a list of patients born in the 1970s, starting from the earliest birth date?

In [43]:
cursor.execute(''' SELECT first_name, last_name, birth_date
                    FROM patients
                    WHERE strftime('%Y', birth_date) BETWEEN '1970' AND '1979'
                    ORDER BY birth_date ASC;  ''')

displayOutput(cursor)

Number of Records -  623
first_name     last_name         birth_date
-------------  ----------------  ------------
Frances        Kobayakawa        1970-01-02
Sunny          Burrell           1970-01-07
Penelope       Beckett           1970-01-14
Deborah        Stewart           1970-01-14
Augusta        Decker            1970-01-22
Sookie         Brearly           1970-02-01
Temple         Wylie             1970-02-10
Deanna         Spano             1970-03-23
Jadu           Principal         1970-03-28
Betty          Stephens          1970-03-28
Jo             Sahid             1970-03-31
Santino        Hawker            1970-04-08
Andy           Guarnaccia        1970-04-10
Richard        Arroway           1970-04-13
Alan           Davis             1970-04-14
Cameron        Spellman          1970-04-21
Bertha         Smart             1970-05-06
Nicky          Klump             1970-05-19
Rose           Petrie            1970-05-27
Kenny          Skelton           1970-05-29
James

#### 29. Can we display patient names in ‘LASTNAME, firstname’ format, sorted by first name ? <ul><li> Example: SMITH,jane </li></ul>

In [44]:
cursor.execute(''' SELECT CONCAT( UPPER(last_name), ',' , LOWER(first_name) ) AS patient_names
                    FROM patients
                    ORDER BY LOWER(first_name) DESC;  ''')

displayOutput(cursor)

Number of Records -  4530
patient_names
---------------------------
MILLER,zoe
CORBIE,ziva
KOBAYAKAWA,zenigata
OVERSTREET,zenigata
BENNETT,zen
MEPHESTO,zelda
MORRIS,zelda
THOMAS,zefram
FLUTE,zefram
TYRELL,zatanna
MARONEY,zatanna
WONG,zane
CHE,zane
DREW,zack
BAKSHI,zachary
PRESTON,yusuke
LAZARUS,yuri
PROVENZA,yuri
CHURCHILL,yuri
RANDALL,yuko
ANNEKE,yuko
VANE,yukari
MCPHERSON,yosh
LYNCH,yosh
BASHIR,woody
RIVIERA,woody
WALLACE,woody
GRANGER,woody
FRANKLIN,woody
CRANE,woodsy
MITCHELL,woodsy
HALE,winnie
CLOCK,winnie
WEASLEY,winnie
LARKIN,winifred
CUNNINGHAM,winifred
JENKINS,winifred
BENNETT,wilson
FOSTER,wilson
LEVINSON,willow
THOREAUXE,willow
AKIMOTO,willow
BOYLE,willow
FRYE,willow
BURDOCK,willow
LOVEGOOD,willow
THIRTEEN,willow
SHAND,willow
ROLAND,willow
HALE,willow
POND,willow
DINGLE,willie
WOLF,willie
MOOREHEAD,willie
WESKER,willie
DUNCAN,willie
TEMPLIN,william
PURBRIGHT,william
ALLEN,william
KENT,william
DE LENA,william
PIKE,william
DRESDEN,william
SNYDER,william
HARRISON,william
FAIRHE

#### 30. Which provinces have a combined patient height of 7,000 or more?

In [45]:
cursor.execute(''' SELECT province_id, SUM(height) AS total_height
                    FROM patients
                    GROUP BY province_id
                    HAVING total_height >= 7000;  ''')

displayOutput(cursor)

Number of Records -  3
province_id      total_height
-------------  --------------
BC                       7720
NS                       9765
ON                     678037


#### 31. Calculate the weight range for patients with the surname 'Maroni'.

In [46]:
cursor.execute(''' SELECT MAX(weight) - MIN(weight) 
                    FROM patients
                    WHERE last_name = 'Maroni'; ''')

displayOutput(cursor)

Number of Records -  1
  MAX(weight) - MIN(weight)
---------------------------
                         71


#### 32. List of all of the days of the month (1-31) and how many admission_dates occurred on that day. Sort by the day with most admissions to least admissions.

In [47]:
cursor.execute(''' SELECT strftime('%d', admission_date) AS day, COUNT(*) AS total_admissions
                    FROM admissions
                    GROUP BY day
                    ORDER BY total_admissions DESC; ''')

displayOutput(cursor) 

Number of Records -  31
  day    total_admissions
-----  ------------------
   11                 184
   04                 184
   09                 183
   02                 180
   12                 179
   06                 179
   16                 177
   21                 174
   28                 173
   13                 173
   07                 167
   18                 166
   08                 166
   03                 166
   30                 165
   20                 165
   15                 164
   14                 164
   10                 164
   24                 161
   05                 161
   17                 160
   25                 159
   26                 157
   23                 156
   01                 156
   19                 152
   27                 149
   29                 140
   22                 140
   31                 103


#### 33. Retrieve details of the latest admission for patient ID 542.

In [48]:
cursor.execute(''' SELECT *
                    FROM admissions
                    GROUP BY patient_id
                    HAVING MAX(admission_date) AND patient_id = 542; ''')

displayOutput(cursor) 

Number of Records -  1
  patient_id  admission_date    discharge_date    diagnosis         attending_doctor_id
------------  ----------------  ----------------  --------------  ---------------------
         542  2019-04-06        2019-04-09        Abdominal Pain                     14


#### 34. List of patient_id, attending_doctor_id, and diagnosis for admissions that match one of the two criteria: <ul> <li> patient_id is an odd number and attending_doctor_id is either 1, 5, or 19.</li> <li> attending_doctor_id contains a 2 and the length of patient_id is 3 characters.</li> </ul>

In [49]:
cursor.execute(''' SELECT patient_id, attending_doctor_id, diagnosis
                    FROM admissions
                    WHERE ( patient_id % 2 != 0 AND attending_doctor_id IN (1, 5, 19) )  OR 
                          ( attending_doctor_id LIKE '%2%' AND LENGTH(patient_id) = 3  ) ; ''')

displayOutput(cursor) 

Number of Records -  663
  patient_id    attending_doctor_id  diagnosis
------------  ---------------------  --------------------------------------------------
           9                     19  Ruptured Appendicitis
          13                      1  Renal Failure
          15                      5  Hiatal Hernia
          31                     19  Cardiovascular Disease
          51                      1  Undiagnosed Chest Pain
         100                     22  Depression, Dementia
         100                     21  Respiratory Failure
         103                     22  Ovarian Cyst
         103                      2  Basal Skull Fracture
         104                     25  Abdominal Pain
         106                      2  Dislocated R/Shoulder Broke L/Leg R/T Car Acc
         108                     12  Unstable Angina
         110                     23  Lumbar Disc Protrusion
         111                      1  Total Hysterectomy
         111                    

#### 35. For each doctor, how many admissions have they attended to date ?

In [50]:
showTableFromDB('doctors')

  doctor_id  first_name    last_name    speciality
-----------  ------------  -----------  -------------------------
          1  Claude        Walls        Internist
          2  Joshua        Green        Cardiologist
          3  Miriam        Tregre       General Surgeon
          4  James         Russo        Obstetrician/Gynecologist
          5  Scott         Hill         Gastroenterologist


In [51]:
showTableFromDB('admissions')

  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-11-06        2018-11-08        Ovarian Dermoid-Cyct                                                   21
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           3  2019-01-24        2019-01-29        Cardiac Arrest                                                          2
           3  2018-10-21        2018-10-27        Congestive Heart Failure                                                8
           6  2018-06-13        2018-06-15        Asthma Exacerbation                                                     3


In [52]:
cursor.execute(''' SELECT d.first_name, d.last_name, COUNT(*) AS total_admissions
                    FROM doctors AS d
                    LEFT JOIN admissions AS a 
                    ON d.doctor_id = a.attending_doctor_id 
                    GROUP BY d.doctor_id; ''')

displayOutput(cursor) 

Number of Records -  27
first_name    last_name      total_admissions
------------  -----------  ------------------
Claude        Walls                       214
Joshua        Green                       187
Miriam        Tregre                      168
James         Russo                       197
Scott         Hill                        179
Tasha         Phillips                    168
Hazel         Patterson                   206
Mickey        Duval                       183
Jon           Nelson                      181
Monica        Singleton                   191
Douglas       Brooks                      201
Flora         Moore                       194
Angelica      Noe                         209
Tyrone        Smart                       182
Marie         Brinkman                    198
Irene         Brooks                      189
Mary          Walker                      191
Bobbi         Estrada                     176
Stephanie     Cohen                       182
Ralph     

#### 36. For each doctor, display their id, full name, and first and last admission date they attended.

In [53]:
showTableFromDB('doctors')

  doctor_id  first_name    last_name    speciality
-----------  ------------  -----------  -------------------------
          1  Claude        Walls        Internist
          2  Joshua        Green        Cardiologist
          3  Miriam        Tregre       General Surgeon
          4  James         Russo        Obstetrician/Gynecologist
          5  Scott         Hill         Gastroenterologist


In [54]:
showTableFromDB('admissions')

  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-11-06        2018-11-08        Ovarian Dermoid-Cyct                                                   21
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           3  2019-01-24        2019-01-29        Cardiac Arrest                                                          2
           3  2018-10-21        2018-10-27        Congestive Heart Failure                                                8
           6  2018-06-13        2018-06-15        Asthma Exacerbation                                                     3


In [55]:
cursor.execute(''' SELECT d.doctor_id, CONCAT(d.first_name,' ', last_name) AS full_name, MIN(a.admission_date) AS first_admission_date, 
                        MAX(a.admission_date) AS last_admission_date
                    FROM doctors AS d
                    LEFT JOIN admissions AS a 
                    ON d.doctor_id = a.attending_doctor_id 
                    GROUP BY d.doctor_id, full_name ; ''')

displayOutput(cursor) 

Number of Records -  27
  doctor_id  full_name         first_admission_date    last_admission_date
-----------  ----------------  ----------------------  ---------------------
          1  Claude Walls      2018-06-10              2019-06-03
          2  Joshua Green      2018-06-06              2019-06-03
          3  Miriam Tregre     2018-06-06              2019-06-03
          4  James Russo       2018-06-10              2019-06-04
          5  Scott Hill        2018-06-06              2019-06-05
          6  Tasha Phillips    2018-06-06              2019-06-02
          7  Hazel Patterson   2018-06-10              2019-06-05
          8  Mickey Duval      2018-06-10              2019-06-05
          9  Jon Nelson        2018-06-10              2019-06-05
         10  Monica Singleton  2018-06-09              2019-06-04
         11  Douglas Brooks    2018-06-10              2019-05-31
         12  Flora Moore       2018-06-07              2019-06-05
         13  Angelica Noe      2

#### 37. What is the count of patients from each province ? Show highest to lowest.

In [56]:
showTableFromDB('province_names')

province_id    province_name
-------------  -------------------------
AB             Alberta
BC             British Columbia
MB             Manitoba
NB             New Brunswick
NL             Newfoundland and Labrador


In [57]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [58]:
cursor.execute(''' SELECT pn.province_name, COUNT(*) AS total_patients
                    FROM province_names AS pn
                    LEFT JOIN patients AS p
                    ON pn.province_id = p.province_id
                    GROUP BY pn.province_name
                    ORDER BY total_patients DESC; ''')

displayOutput(cursor) 

Number of Records -  13
province_name                total_patients
-------------------------  ----------------
Ontario                                4298
Nova Scotia                              60
British Columbia                         49
Alberta                                  47
Manitoba                                 31
Quebec                                   23
Saskatchewan                             13
Newfoundland and Labrador                 9
Yukon                                     1
Prince Edward Island                      1
Nunavut                                   1
Northwest Territories                     1
New Brunswick                             1


#### 38. Could you list each admission with the patient’s full name, diagnosis, and doctor’s name ?

In [59]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [60]:
showTableFromDB('admissions')

  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-11-06        2018-11-08        Ovarian Dermoid-Cyct                                                   21
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           3  2019-01-24        2019-01-29        Cardiac Arrest                                                          2
           3  2018-10-21        2018-10-27        Congestive Heart Failure                                                8
           6  2018-06-13        2018-06-15        Asthma Exacerbation                                                     3


In [61]:
showTableFromDB('doctors')

  doctor_id  first_name    last_name    speciality
-----------  ------------  -----------  -------------------------
          1  Claude        Walls        Internist
          2  Joshua        Green        Cardiologist
          3  Miriam        Tregre       General Surgeon
          4  James         Russo        Obstetrician/Gynecologist
          5  Scott         Hill         Gastroenterologist


In [62]:
cursor.execute(''' SELECT CONCAT(p.first_name, ' ', p.last_name) AS patient_full_name, a.diagnosis,
                          CONCAT(d.first_name, ' ', d.last_name) AS doctor_name
                    FROM patients AS p
                    INNER JOIN admissions AS a
                    ON p.patient_id = a.patient_id
                    LEFT JOIN doctors AS d
                    ON a.attending_doctor_id = d.doctor_id; ''')

displayOutput(cursor) 

Number of Records -  5067
patient_full_name          diagnosis                                           doctor_name
-------------------------  --------------------------------------------------  ----------------
Donald Waterfield          Ovarian Dermoid-Cyct                                Lisa Cuddy
Donald Waterfield          Ineffective Breathin Pattern R/T Fluid Accumulatio  Jenny Pulaski
Jiji Sharma                Cardiac Arrest                                      Joshua Green
Jiji Sharma                Congestive Heart Failure                            Mickey Duval
Sue Falcon                 Asthma Exacerbation                                 Miriam Tregre
Sue Falcon                 Uterine Fibroid                                     Simon Santiago
Thomas ONeill              Cancer                                              Mickey Duval
Sonny Beckett              Amigima                                             Tasha Phillips
Sister Spitzer             Osteoarthritis      

#### 39. Are there any duplicate patients based on first and last names ?

In [63]:
cursor.execute(''' SELECT first_name, last_name, COUNT(*) AS duplicate_patients
                    FROM patients
                    GROUP BY first_name, last_name
                    HAVING COUNT(*) > 1; ''')

displayOutput(cursor)

Number of Records -  19
first_name    last_name      duplicate_patients
------------  -----------  --------------------
Alexandra     Holroyd                         2
Avon          Travis                          2
Belladonna    Thomas                          2
Chris         Johnson                         2
Dol           Forelli                         2
Emil          Owens                           2
Jack          Bennett                         2
Jack          McArthur                        2
Jack          Valentine                       2
Jackson       Kennedy                         2
Jim           Grant                           2
Kay           Sanderson                       2
Luanne        Matthews                        2
Marcus        Jackson                         2
Mendy         Farrell                         2
Sam           Carr                            2
Sam           Jackson                         2
Sam           Moss                            2
Samantha      Bl

#### 40. List of patient’s names with their height (cm to feet), weight (KG to pounds), birth date, and full gender name.

In [64]:
cursor.execute(''' SELECT first_name, last_name, birth_date,
                        ROUND(height / 30.48, 1) AS height_in_feet, ROUND(weight * 2.205, 0) AS weight_in_kg, 
                        CASE
                            WHEN gender = 'M' THEN 'Male'
                            WHEN gender = 'F' THEN 'Female'
                        END AS role
                    FROM patients; ''')

displayOutput(cursor)

Number of Records -  4530
first_name     last_name         birth_date      height_in_feet    weight_in_kg  role
-------------  ----------------  ------------  ----------------  --------------  ------
Donald         Waterfield        1963-02-12                 5.1             143  Male
Mickey         Baasha            1981-05-28                 6.1             168  Male
Jiji           Sharma            1957-09-05                 6.4             234  Male
Blair          Diaz              1967-01-07                 6.3             229  Male
Charles        Wolfe             2017-11-19                 1.5              22  Male
Sue            Falcon            2017-09-30                 1.4              11  Female
Thomas         ONeill            1993-01-31                 5.9             258  Male
Sonny          Beckett           1952-12-11                 5.7             232  Male
Sister         Spitzer           1966-10-15                 5.7             209  Female
Cedric         Coltran

#### 41. Which patients don’t have any admission records ?

In [65]:
cursor.execute(''' SELECT patient_id, first_name, last_name
                    FROM patients 
                    WHERE patient_id NOT IN ( SELECT patient_id FROM admissions ) ; ''')

displayOutput(cursor)

Number of Records -  1148
  patient_id  first_name     last_name
------------  -------------  ----------------
           2  Mickey         Baasha
           4  Blair          Diaz
           5  Charles        Wolfe
          14  Rick           Bennett
          19  John           West
          25  John           Farley
          26  Nea            Kobayakawa
          30  Roderick       Payne
          42  Alice          Donovan
          43  Stone          Cutting
          44  Neil           Allan
          50  Phil           Beckett
          55  Rose           Spellman
          57  Woody          Riviera
          65  Haiku          Alpert
          72  Lily           Vargas
          80  Ernest         Clarkson
          85  Phoebe         Bell
          89  Sam            Spratt
          95  Gene           Edwards
          96  Jonny          Christian
          99  Richard        Davis
         107  Gillian        Dyson
         109  William        Templin
         116  Mabl

#### 42. Could we categorize patients by weight groups ? Show the count of patients in each group. <ul><li>For example, if they weight 100 to 109 they are placed in the 100 weight group, 110-119 = 110 weight group, etc.</li></ul>

In [66]:
cursor.execute(''' SELECT 
                        FLOOR(weight / 10) * 10 AS weight_group,
                        COUNT(*) AS total_patients
                    FROM patients
                    GROUP BY weight_group
                    ORDER BY weight_group DESC; ''')

displayOutput(cursor)

Number of Records -  15
  weight_group    total_patients
--------------  ----------------
           140                 6
           130                59
           120               191
           110               426
           100               507
            90               403
            80               478
            70               633
            60               685
            50               443
            40               202
            30               126
            20               165
            10               114
             0                92


#### 43. List of patient_id, weight, height, isObese from the patients table. <ul><li> Display isObese as a boolean 0 or 1.</li><li>Obese is defined as weight (kg) / ( height (m)2 ) >= 30.</li></ul>


In [67]:
cursor.execute(''' SELECT patient_id, weight, height, 
                    CASE
                        WHEN ( weight / (height/100.0 * height/100.0) ) >= 30 THEN 1
                        ELSE 0
                    END AS isObese
                    FROM patients; ''') 

displayOutput(cursor)

Number of Records -  4530
  patient_id    weight    height    isObese
------------  --------  --------  ---------
           1        65       156          0
           2        76       185          0
           3       106       194          0
           4       104       191          0
           5        10        47          1
           6         5        43          0
           7       117       180          1
           8       105       174          1
           9        95       173          1
          10        61       157          0
          11        74       158          0
          12        46       145          0
          13        77       146          1
          14        95       220          0
          15        72       172          0
          16        59       153          0
          17       114       179          1
          18        95       163          1
          19        61       138          1
          20       116       194          1
      

#### 44. Could you show patients diagnosed with ‘Epilepsy’ by Dr. Lisa and include her specialty ?

In [68]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [69]:
showTableFromDB('admissions')

  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-11-06        2018-11-08        Ovarian Dermoid-Cyct                                                   21
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           3  2019-01-24        2019-01-29        Cardiac Arrest                                                          2
           3  2018-10-21        2018-10-27        Congestive Heart Failure                                                8
           6  2018-06-13        2018-06-15        Asthma Exacerbation                                                     3


In [70]:
showTableFromDB('doctors')

  doctor_id  first_name    last_name    speciality
-----------  ------------  -----------  -------------------------
          1  Claude        Walls        Internist
          2  Joshua        Green        Cardiologist
          3  Miriam        Tregre       General Surgeon
          4  James         Russo        Obstetrician/Gynecologist
          5  Scott         Hill         Gastroenterologist


In [71]:
cursor.execute(''' SELECT p.patient_id AS patient_id, p.first_name AS patient_first_name, p.last_name AS patient_last_name, 
                        d.speciality AS attending_doctor_speciality
                    FROM patients AS p
                    INNER JOIN admissions AS a
                    ON p.patient_id = a.patient_id
                    INNER JOIN doctors AS d
                    ON a.attending_doctor_id = d.doctor_id
                    WHERE a.diagnosis = 'Epilepsy' AND d.first_name = 'Lisa'; ''') 

displayOutput(cursor)

Number of Records -  2
  patient_id  patient_first_name    patient_last_name    attending_doctor_speciality
------------  --------------------  -------------------  -----------------------------
         468  Frank                 Anderson             Obstetrician/Gynecologist
         701  Precious              Ashton               Obstetrician/Gynecologist


#### 45. All patients who have gone through admissions, can see their medical documents on our site. Those patients are given a temporary password after their first admission. Show the patient_id and temp_password. The password must be the following, in order :<ul><li>patient_id</li><li>The numerical length of patient's last_name</li><li>Year of patient's birth_date</li></ul>

In [72]:
cursor.execute(''' SELECT p.patient_id, CONCAT( p.patient_id, LENGTH(p.last_name), strftime('%Y', p.birth_date) ) AS temp_password
                    FROM patients AS p
                    RIGHT JOIN admissions AS a
                    ON p.patient_id = a.patient_id
                    GROUP BY p.patient_id; ''') 

displayOutput(cursor)

Number of Records -  3382
  patient_id    temp_password
------------  ---------------
           1          1101963
           3           361957
           6           662017
           7           761993
           8           871952
           9           971966
          10          1081961
          11          1171969
          12          1281949
          13          1391954
          15          1551977
          16          1661951
          17          1791987
          18         18121966
          20          2071951
          21          2171975
          22          2241975
          23          2361998
          24          2472001
          27          2791964
          28          2871953
          29          2961986
          31          3161982
          32          3272009
          33          3341967
          34          3462002
          35          3572011
          36          3681974
          37          3771996
          38          3882010
          39  

#### 46. Each admission costs $50 for patients without insurance, and $10 for patients with insurance. All patients with an even patient_id have insurance. <br>Give each patient a 'Yes' if they have insurance, and a 'No' if they don't have insurance. Add up the admission_total cost for each has_insurance group.


In [73]:
cursor.execute(''' SELECT 
                        CASE
                            WHEN patient_id % 2 = 0 THEN 'YES'
                            ELSE 'NO'
                        END AS has_insurance,
                        SUM(CASE
                                WHEN patient_id % 2 = 0 THEN 10
                                ELSE 50
                            END) AS cost_after_insurance 
                    FROM admissions
                    GROUP BY has_insurance; ''') 

displayOutput(cursor)

Number of Records -  2
has_insurance      cost_after_insurance
---------------  ----------------------
NO                               127800
YES                               25110


#### 47. Which provinces have more male than female patients ? Show the province names.

In [74]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [75]:
showTableFromDB('province_names')

province_id    province_name
-------------  -------------------------
AB             Alberta
BC             British Columbia
MB             Manitoba
NB             New Brunswick
NL             Newfoundland and Labrador


In [76]:
cursor.execute(''' SELECT pn.province_name
                    FROM patients AS p
                    INNER JOIN province_names AS pn
                    ON p.province_id = pn.province_id
                    GROUP BY province_name
                    HAVING 
                        COUNT( CASE WHEN gender = 'M' THEN 1 END ) > COUNT( CASE WHEN gender = 'F' THEN 1 END ); ''') 

displayOutput(cursor)

Number of Records -  7
province_name
-------------------------
Alberta
British Columbia
Manitoba
Newfoundland and Labrador
Nova Scotia
Ontario
Saskatchewan


#### 48. We are looking for a specific patient. Pull all columns for the patient who matches the following criteria: <br><ul><li>first_name contains an 'r' after the first two letters</li><li>Identifies their gender as 'F'</li><li>Born in February, May, or December</li><li>Their weight would be between 60kg and 80kg</li><li>Their patient_id is an odd number</li><li>They are from the city 'Kingston'</li></ul>

In [77]:
cursor.execute(''' SELECT * 
                    FROM patients
                    WHERE first_name LIKE '__r%' 
                        AND gender = 'F' 
                        AND strftime('%m', birth_date) IN ('2', '5', '12') 
                        AND weight BETWEEN 60 AND 80 
                        AND patient_id % 2 != 0 
                        AND city = 'Kingston' ; ''') 

displayOutput(cursor)

Number of Records -  1
  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
        1839  Chris         Cardenas     F         1969-12-02    Kingston  ON             NKA               141        71


#### 49. What percent of our patients are male, rounded to the nearest whole number ?

In [78]:
cursor.execute(''' SELECT 
                        ROUND(
                                (SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 
                                2
                            ) || '%' AS percent_of_male_patients
                    FROM 
                    patients; ''') 

displayOutput(cursor)

Number of Records -  1
percent_of_male_patients
--------------------------
54.48%


#### 50. For each day display the total amount of admissions on that day. Display the amount changed from the previous date.

In [79]:
cursor.execute(''' SELECT admission_date,
                        COUNT(*) AS total_admissions,
                        COUNT(*) - LAG(COUNT(*)) OVER (ORDER BY admission_date) AS change_from_previous_day
                    FROM admissions
                    GROUP BY admission_date
                    ORDER BY admission_date; ''') 

displayOutput(cursor)

Number of Records -  365
admission_date      total_admissions    change_from_previous_day
----------------  ------------------  --------------------------
2018-06-06                        17
2018-06-07                         9                          -8
2018-06-08                         9                           0
2018-06-09                        18                           9
2018-06-10                        12                          -6
2018-06-11                        22                          10
2018-06-12                         9                         -13
2018-06-13                        12                           3
2018-06-14                        14                           2
2018-06-15                        11                          -3
2018-06-16                        13                           2
2018-06-17                        20                           7
2018-06-18                        17                          -3
2018-06-19                  

#### 51. List all province names alphabetically, placing Ontario at the top.

In [80]:
cursor.execute(''' SELECT province_name
                    FROM province_names
                    ORDER BY 
                            CASE 
                                WHEN province_name = 'Ontario' THEN 0 
                                ELSE 1 
                            END,
                            province_name; ''') 

displayOutput(cursor)

Number of Records -  13
province_name
-------------------------
Ontario
Alberta
British Columbia
Manitoba
New Brunswick
Newfoundland and Labrador
Northwest Territories
Nova Scotia
Nunavut
Prince Edward Island
Quebec
Saskatchewan
Yukon


#### 52. We need a breakdown for the total amount of admissions each doctor has started each year. Show the doctor_id, doctor_full_name, specialty, year, total_admissions for that year.

In [81]:
showTableFromDB('doctors')

  doctor_id  first_name    last_name    speciality
-----------  ------------  -----------  -------------------------
          1  Claude        Walls        Internist
          2  Joshua        Green        Cardiologist
          3  Miriam        Tregre       General Surgeon
          4  James         Russo        Obstetrician/Gynecologist
          5  Scott         Hill         Gastroenterologist


In [82]:
showTableFromDB('admissions')

  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-11-06        2018-11-08        Ovarian Dermoid-Cyct                                                   21
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           3  2019-01-24        2019-01-29        Cardiac Arrest                                                          2
           3  2018-10-21        2018-10-27        Congestive Heart Failure                                                8
           6  2018-06-13        2018-06-15        Asthma Exacerbation                                                     3


In [83]:
cursor.execute(''' SELECT d.doctor_id, CONCAT(d.first_name, ' ', d.last_name) AS doctor_name, d.speciality, 
                        strftime('%Y', a.admission_date) AS selected_year, COUNT(*) AS total_admissions
                    FROM doctors as d
                    LEFT JOIN admissions as a
                    ON d.doctor_id = a.attending_doctor_id
                    GROUP BY d.doctor_id, doctor_name, d.speciality, selected_year; ''') 

export_query_to_excel(cursor, 'Output.xlsx')

Data Exported Successfully to Output.xlsx


### Close databse connection

In [84]:
cursor.close()
connection.close()

### Conclusion

#### <div style="text-align: justify;"> This comprehensive analysis of patient and admissions data will equip the hospital management team with actionable insights on patient demographics, admission patterns, and physician performance. By understanding patient distributions, identifying repeat admissions, and evaluating doctor engagement, the hospital can better allocate resources, improve care quality, and optimize operational efficiencies. </div>